# רשתות עצביות חוזרות

במודול הקודם השתמשנו בייצוגים סמנטיים עשירים של טקסט ובמסווג ליניארי פשוט מעל ההטמעות. מה שהארכיטקטורה הזו עושה הוא לתפוס את המשמעות המצטברת של מילים במשפט, אך היא אינה מתחשבת ב**סדר** המילים, מכיוון שפעולת האגרגציה מעל ההטמעות הסירה את המידע הזה מהטקסט המקורי. מכיוון שמודלים אלו אינם מסוגלים לדגם את סדר המילים, הם אינם יכולים לפתור משימות מורכבות או עמומות יותר כמו יצירת טקסט או מענה על שאלות.

כדי לתפוס את המשמעות של רצף טקסט, עלינו להשתמש בארכיטקטורה אחרת של רשת עצבית, הנקראת **רשת עצבית חוזרת**, או RNN. ב-RNN, אנו מעבירים את המשפט דרך הרשת סמל אחד בכל פעם, והרשת מייצרת **מצב** מסוים, אותו אנו מעבירים שוב לרשת יחד עם הסמל הבא.

בהינתן רצף קלט של טוקנים $X_0,\dots,X_n$, RNN יוצרת רצף של בלוקים של רשת עצבית, ומאמנת את הרצף הזה מקצה לקצה באמצעות התפשטות לאחור. כל בלוק רשת מקבל זוג $(X_i,S_i)$ כקלט, ומייצר $S_{i+1}$ כתוצאה. המצב הסופי $S_n$ או הפלט $X_n$ מועבר למסווג ליניארי כדי לייצר את התוצאה. כל בלוקי הרשת חולקים את אותם משקלים, ומאומנים מקצה לקצה באמצעות מעבר אחד של התפשטות לאחור.

מכיוון שוקטורי המצב $S_0,\dots,S_n$ מועברים דרך הרשת, היא מסוגלת ללמוד את התלויות הרציפות בין מילים. לדוגמה, כאשר המילה *לא* מופיעה במקום כלשהו ברצף, היא יכולה ללמוד לשלול אלמנטים מסוימים בתוך וקטור המצב, מה שמוביל לשלילה.

> מכיוון שמשקלי כל בלוקי ה-RNN בתמונה משותפים, ניתן לייצג את אותה תמונה כבלוק אחד (מימין) עם לולאת משוב חוזרת, שמעבירה את מצב הפלט של הרשת חזרה לקלט.

בואו נראה כיצד רשתות עצביות חוזרות יכולות לעזור לנו לסווג את מערך הנתונים של החדשות שלנו.


In [1]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...
Building vocab...


## מסווג RNN פשוט

במקרה של RNN פשוט, כל יחידה חוזרת היא רשת ליניארית פשוטה, אשר מקבלת וקטור קלט משולב ווקטור מצב, ומייצרת וקטור מצב חדש. PyTorch מייצגת יחידה זו באמצעות מחלקת `RNNCell`, ורשתות של יחידות כאלה - כשכבת `RNN`.

כדי להגדיר מסווג RNN, תחילה ניישם שכבת הטמעה להורדת הממדיות של אוצר המילים בקלט, ולאחר מכן נוסיף שכבת RNN מעליה:


In [2]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

> **Note:** אנו משתמשים בשכבת הטמעה לא מאומנת כאן לצורך פשטות, אך לתוצאות טובות יותר ניתן להשתמש בשכבת הטמעה מאומנת מראש עם הטמעות Word2Vec או GloVe, כפי שתואר ביחידה הקודמת. כדי להבין טוב יותר, ייתכן שתרצו להתאים את הקוד כך שיעבוד עם הטמעות מאומנות מראש.

במקרה שלנו, נשתמש במעמיס נתונים מרופד, כך שכל אצווה תכיל מספר רצפים מרופדים באורך זהה. שכבת ה-RNN תקבל את רצף טנסורי ההטמעות ותפיק שני פלטים:
* $x$ הוא רצף של פלטי תאי RNN בכל שלב
* $h$ הוא מצב מוסתר סופי עבור האלמנט האחרון ברצף

לאחר מכן ניישם מסווג ליניארי מחובר-מלא כדי לקבל את מספר המחלקות.

> **Note:** RNNs הם די קשים לאימון, מכיוון שברגע שתאי ה-RNN נפרסים לאורך אורך הרצף, מספר השכבות המעורבות באחזור לאחור הופך להיות די גדול. לכן, יש לבחור קצב למידה קטן ולאמן את הרשת על מערך נתונים גדול יותר כדי להפיק תוצאות טובות. זה יכול לקחת זמן רב, ולכן מומלץ להשתמש ב-GPU.


In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.3090625
6400: acc=0.38921875
9600: acc=0.4590625
12800: acc=0.511953125
16000: acc=0.5506875
19200: acc=0.57921875
22400: acc=0.6070089285714285
25600: acc=0.6304296875
28800: acc=0.6484027777777778
32000: acc=0.66509375
35200: acc=0.6790056818181818
38400: acc=0.6929166666666666
41600: acc=0.7035817307692308
44800: acc=0.7137276785714286
48000: acc=0.72225
51200: acc=0.73001953125
54400: acc=0.7372794117647059
57600: acc=0.7436631944444444
60800: acc=0.7503947368421052
64000: acc=0.75634375
67200: acc=0.7615773809523809
70400: acc=0.7662642045454545
73600: acc=0.7708423913043478
76800: acc=0.7751822916666666
80000: acc=0.7790625
83200: acc=0.7825
86400: acc=0.7858564814814815
89600: acc=0.7890513392857142
92800: acc=0.7920474137931034
96000: acc=0.7952708333333334
99200: acc=0.7982258064516129
102400: acc=0.80099609375
105600: acc=0.8037594696969697
108800: acc=0.8060569852941176


## זיכרון לטווח ארוך וקצר (LSTM)

אחת הבעיות המרכזיות של RNNs קלאסיים היא בעיית **הגרדיאנטים הנעלמים**. מכיוון ש-RNNs מאומנים מקצה לקצה במעבר אחד של back-propagation, קשה להם להעביר את השגיאה לשכבות הראשונות של הרשת, ולכן הרשת אינה יכולה ללמוד קשרים בין טוקנים רחוקים. אחת הדרכים להימנע מבעיה זו היא להכניס **ניהול מצב מפורש** באמצעות שימוש במה שנקרא **שערים**. ישנן שתי ארכיטקטורות ידועות מסוג זה: **זיכרון לטווח ארוך וקצר** (LSTM) ו-**יחידת ממסר עם שערים** (GRU).

![תמונה המציגה דוגמה של תא זיכרון לטווח ארוך וקצר](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

רשת LSTM מאורגנת בצורה דומה ל-RNN, אך ישנם שני מצבים שעוברים משכבה לשכבה: המצב האמיתי $c$, והווקטור הנסתר $h$. בכל יחידה, וקטור נסתר $h_i$ משולב עם הקלט $x_i$, והם שולטים במה שקורה למצב $c$ באמצעות **שערים**. כל שער הוא רשת נוירונים עם פונקציית הפעלה מסוג סיגמואיד (פלט בטווח $[0,1]$), שניתן לחשוב עליה כמסכה ביטית כאשר מכפילים אותה בווקטור המצב. השערים הם (משמאל לימין בתמונה למעלה):
* **שער השכחה** לוקח את הווקטור הנסתר וקובע אילו רכיבים של הווקטור $c$ עלינו לשכוח ואילו להעביר הלאה.
* **שער הקלט** לוקח מידע מסוים מהקלט ומהווקטור הנסתר, ומכניס אותו למצב.
* **שער הפלט** משנה את המצב דרך שכבה ליניארית עם הפעלה מסוג $\tanh$, ואז בוחר חלק מהרכיבים שלה באמצעות הווקטור הנסתר $h_i$ כדי לייצר מצב חדש $c_{i+1}$.

ניתן לחשוב על רכיבי המצב $c$ כדגלים שניתן להדליק ולכבות. לדוגמה, כאשר אנו נתקלים בשם *אליס* ברצף, ייתכן שנרצה להניח שהוא מתייחס לדמות נשית, ולהרים דגל במצב שמציין שיש לנו שם עצם נשי במשפט. כאשר ניתקל בהמשך בביטוי *וגם טום*, נרים דגל שמציין שיש לנו שם עצם ברבים. כך, על ידי מניפולציה של המצב, נוכל לכאורה לעקוב אחר תכונות דקדוקיות של חלקי המשפט.

> **Note**: משאב מצוין להבנת המבנה הפנימי של LSTM הוא המאמר הנהדר הזה [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) מאת כריסטופר אולה.

למרות שהמבנה הפנימי של תא LSTM עשוי להיראות מורכב, PyTorch מסתירה את המימוש הזה בתוך המחלקה `LSTMCell`, ומספקת את האובייקט `LSTM` לייצוג שכבת LSTM שלמה. לכן, המימוש של מסווג LSTM יהיה דומה מאוד ל-RNN הפשוט שראינו קודם:


In [4]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

In [5]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.259375
6400: acc=0.25859375
9600: acc=0.26177083333333334
12800: acc=0.2784375
16000: acc=0.313
19200: acc=0.3528645833333333
22400: acc=0.3965625
25600: acc=0.4385546875
28800: acc=0.4752777777777778
32000: acc=0.505375
35200: acc=0.5326704545454546
38400: acc=0.5557552083333334
41600: acc=0.5760817307692307
44800: acc=0.5954910714285714
48000: acc=0.6118333333333333
51200: acc=0.62681640625
54400: acc=0.6404779411764706
57600: acc=0.6520138888888889
60800: acc=0.662828947368421
64000: acc=0.673546875
67200: acc=0.6831547619047619
70400: acc=0.6917897727272727
73600: acc=0.6997146739130434
76800: acc=0.707109375
80000: acc=0.714075
83200: acc=0.7209134615384616
86400: acc=0.727037037037037
89600: acc=0.7326674107142858
92800: acc=0.7379633620689655
96000: acc=0.7433645833333333
99200: acc=0.7479032258064516
102400: acc=0.752119140625
105600: acc=0.7562405303030303
108800: acc=0.76015625
112000: acc=0.7641339285714286
115200: acc=0.7677777777777778
118400: acc=0.77112331081

(0.03487814127604167, 0.7728)

## רצפים ארוזים

בדוגמה שלנו, היינו צריכים למלא את כל הרצפים במיני-אצווה עם וקטורי אפס. למרות שזה גורם לבזבוז זיכרון מסוים, עם RNNs זה קריטי יותר מכיוון שנוצרים תאי RNN נוספים עבור פריטי הקלט המלאים, שמשתתפים באימון אך אינם נושאים מידע קלט חשוב. יהיה הרבה יותר טוב לאמן את ה-RNN רק לגודל הרצף האמיתי.

כדי לעשות זאת, פורמט מיוחד לאחסון רצפים מלאים הוצג ב-PyTorch. נניח שיש לנו מיני-אצווה מלאה שנראית כך:
```
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
```
כאן 0 מייצג ערכים מלאים, והווקטור של אורכי הרצפים האמיתיים הוא `[5,3,1]`.

כדי לאמן RNN בצורה יעילה עם רצפים מלאים, אנחנו רוצים להתחיל את האימון של קבוצת תאי RNN הראשונה עם מיני-אצווה גדולה (`[1,6,9]`), אבל אז לסיים את העיבוד של הרצף השלישי ולהמשיך את האימון עם מיני-אצוות קטנות יותר (`[2,7]`, `[3,8]`), וכן הלאה. כך, רצף ארוז מיוצג כווקטור אחד - במקרה שלנו `[1,6,9,2,7,3,8,4,5]`, וווקטור אורכים (`[5,3,1]`), שממנו ניתן לשחזר בקלות את המיני-אצווה המלאה המקורית.

כדי ליצור רצף ארוז, ניתן להשתמש בפונקציה `torch.nn.utils.rnn.pack_padded_sequence`. כל השכבות החוזרות, כולל RNN, LSTM ו-GRU, תומכות ברצפים ארוזים כקלט ומייצרות פלט ארוז, שניתן לפענח באמצעות `torch.nn.utils.rnn.pad_packed_sequence`.

כדי להיות מסוגלים ליצור רצף ארוז, אנחנו צריכים להעביר את וקטור האורכים לרשת, ולכן אנחנו צריכים פונקציה שונה להכנת מיני-אצוות:


In [6]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

הרשת בפועל תהיה דומה מאוד ל-`LSTMClassifier` שהוזכר למעלה, אך מעבר ה-`forward` יקבל גם מיני-אצווה מרופדת וגם וקטור של אורכי הרצפים. לאחר חישוב ההטמעה, אנו מחשבים רצף ארוז, מעבירים אותו לשכבת ה-LSTM, ואז פותחים את התוצאה בחזרה.

> **הערה**: למעשה, אנחנו לא משתמשים בתוצאה הבלתי-ארוזה `x`, מכיוון שאנחנו משתמשים בפלט מהשכבות הנסתרות בחישובים הבאים. לכן, ניתן להסיר את שלב פתיחת האריזה לחלוטין מהקוד הזה. הסיבה שאנחנו משאירים אותו כאן היא כדי לאפשר לך לשנות את הקוד בקלות, במקרה שתצטרך להשתמש בפלט הרשת בחישובים נוספים.


In [7]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        return self.fc(h[-1])

In [8]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


3200: acc=0.285625
6400: acc=0.33359375
9600: acc=0.3876041666666667
12800: acc=0.44078125
16000: acc=0.4825
19200: acc=0.5235416666666667
22400: acc=0.5559821428571429
25600: acc=0.58609375
28800: acc=0.6116666666666667
32000: acc=0.63340625
35200: acc=0.6525284090909091
38400: acc=0.668515625
41600: acc=0.6822596153846154
44800: acc=0.6948214285714286
48000: acc=0.7052708333333333
51200: acc=0.71521484375
54400: acc=0.7239889705882353
57600: acc=0.7315277777777778
60800: acc=0.7388486842105263
64000: acc=0.74571875
67200: acc=0.7518303571428572
70400: acc=0.7576988636363636
73600: acc=0.7628940217391305
76800: acc=0.7681510416666667
80000: acc=0.7728125
83200: acc=0.7772235576923077
86400: acc=0.7815393518518519
89600: acc=0.7857700892857142
92800: acc=0.7895043103448276
96000: acc=0.7930520833333333
99200: acc=0.7959072580645161
102400: acc=0.798994140625
105600: acc=0.802064393939394
108800: acc=0.8051378676470589
112000: acc=0.8077857142857143
115200: acc=0.8104600694444445
118400

(0.029785829671223958, 0.8138166666666666)

> **הערה:** ייתכן ששמת לב לפרמטר `use_pack_sequence` שאנו מעבירים לפונקציית האימון. נכון לעכשיו, פונקציית `pack_padded_sequence` דורשת שמערך אורך הרצף יהיה על התקן CPU, ולכן פונקציית האימון צריכה להימנע מהעברת נתוני אורך הרצף ל-GPU במהלך האימון. ניתן לעיין במימוש של פונקציית `train_emb` בקובץ [`torchnlp.py`](../../../../../lessons/5-NLP/16-RNN/torchnlp.py).


## RNN דו-כיווני ורב-שכבתי

בדוגמאות שלנו, כל הרשתות החוזרות פעלו בכיוון אחד, מתחילת הרצף ועד סופו. זה נראה טבעי, כי זה דומה לאופן שבו אנו קוראים ומאזינים לדיבור. עם זאת, במקרים מעשיים רבים יש לנו גישה אקראית לרצף הקלט, ולכן ייתכן שיהיה הגיוני להריץ חישוב חוזר בשני הכיוונים. רשתות כאלה נקראות **RNN דו-כיווני**, וניתן ליצור אותן על ידי העברת הפרמטר `bidirectional=True` לבנאי של RNN/LSTM/GRU.

כאשר עובדים עם רשת דו-כיוונית, נצטרך שני וקטורי מצב חבוי, אחד לכל כיוון. PyTorch מקודד את הווקטורים הללו כווקטור אחד בגודל כפול, מה שמאוד נוח, כי בדרך כלל תעבירו את מצב החבוי המתקבל לשכבה לינארית מחוברת-מלאה, ותצטרכו רק לקחת את הגידול בגודל בחשבון בעת יצירת השכבה.

רשת חוזרת, בין אם היא חד-כיוונית או דו-כיוונית, לוכדת דפוסים מסוימים בתוך רצף, ויכולה לאחסן אותם בווקטור מצב או להעביר אותם לפלט. כמו ברשתות קונבולוציה, ניתן לבנות שכבה חוזרת נוספת מעל הראשונה כדי ללכוד דפוסים ברמה גבוהה יותר, שנבנים מדפוסים ברמה נמוכה שהופקו על ידי השכבה הראשונה. זה מוביל אותנו למושג של **RNN רב-שכבתי**, שמורכב משתי רשתות חוזרות או יותר, כאשר הפלט של השכבה הקודמת מועבר לשכבה הבאה כקלט.

![תמונה המציגה RNN רב-שכבתי מסוג LSTM](../../../../../translated_images/he/multi-layer-lstm.dd975e29bb2a59fe.webp)

*תמונה מתוך [הפוסט הנהדר הזה](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) מאת Fernando López*

PyTorch הופך את בניית הרשתות הללו למשימה פשוטה, כי כל מה שצריך לעשות הוא להעביר את הפרמטר `num_layers` לבנאי של RNN/LSTM/GRU כדי לבנות באופן אוטומטי מספר שכבות של חזרתיות. זה גם אומר שגודל וקטור החבוי/מצב יגדל באופן יחסי, ותצטרכו לקחת זאת בחשבון בעת טיפול בפלט של השכבות החוזרות.


## רשתות עצביות חוזרות (RNNs) למשימות אחרות

ביחידה זו ראינו ש-RNNs יכולים לשמש לסיווג רצפים, אך למעשה, הם יכולים להתמודד עם משימות רבות נוספות, כמו יצירת טקסט, תרגום מכונה ועוד. נבחן את המשימות הללו ביחידה הבאה.



---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש לקחת בחשבון שתרגומים אוטומטיים עשויים להכיל שגיאות או אי דיוקים. המסמך המקורי בשפתו המקורית צריך להיחשב כמקור סמכותי. עבור מידע קריטי, מומלץ להשתמש בתרגום מקצועי על ידי אדם. איננו נושאים באחריות לאי הבנות או לפרשנויות שגויות הנובעות משימוש בתרגום זה.
